In [1]:
import requests
import pandas as pd
import json

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [5]:
# --- 第一步：我们需要稍微修改一下爬虫函数，让它支持手动指定 secid ---
# (不需要重新导入库，只要运行这个覆盖之前的 get_k_history 即可)

def get_k_history_fixed(code, secid_override=None, limit=20):
    """
    修改版 K线获取函数，支持强制指定 secid
    """
    # 如果指定了 secid (比如指数)，就用指定的；否则用自动判断的
    if secid_override:
        secid = secid_override
    else:
        secid = get_secid(code)
    
    url = "http://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        "secid": secid,
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "fields1": "f1,f2,f3,f4,f5,f6",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61", 
        "klt": "101", "fqt": "1", "end": "20500101", "lmt": limit,
    }
    
    try:
        response = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"})
        data_json = response.json()
        if not (data_json and data_json.get('data') and data_json['data'].get('klines')):
            return None
            
        klines = data_json['data']['klines']
        rows = [line.split(',') for line in klines]
        df = pd.DataFrame(rows, columns=['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率'])
        
        # 转换数据类型
        df['日期'] = pd.to_datetime(df['日期'])
        df['成交额'] = pd.to_numeric(df['成交额'], errors='coerce')
        return df
    except Exception:
        return None

# --- 第二步：运行修正后的统计函数 ---

def get_recent_market_turnover_fixed(days=7):
    print(f"正在获取最近 {days} 天的全市场（修正版）成交数据...")

    # 关键修改：手动指定指数的 secid，确保获取的是大盘指数，而不是个股
    # 1.000001 = 上证指数 (上海大盘)
    # 0.399001 = 深证成指 (深圳大盘)
    # 0.899050 = 北证50   (北交所代表)
    sh_df = get_k_history_fixed('000001', secid_override='1.000001', limit=20) 
    sz_df = get_k_history_fixed('399001', secid_override='0.399001', limit=20)
    bj_df = get_k_history_fixed('899050', secid_override='0.899050', limit=20)
    
    if sh_df is None or sz_df is None or bj_df is None:
        print("数据获取失败，请检查网络。")
        return

    # 数据重命名与合并
    sh = sh_df[['日期', '成交额']].rename(columns={'成交额': '沪市'})
    sz = sz_df[['日期', '成交额']].rename(columns={'成交额': '深市'})
    bj = bj_df[['日期', '成交额']].rename(columns={'成交额': '北证'})

    # 合并数据
    df = pd.merge(sh, sz, on='日期').merge(bj, on='日期')

    # 计算总成交 (单位转为亿)
    df['沪市(亿)'] = df['沪市'] / 1e8
    df['深市(亿)'] = df['深市'] / 1e8
    df['北证(亿)'] = df['北证'] / 1e8
    df['总成交(亿)'] = df['沪市(亿)'] + df['深市(亿)'] + df['北证(亿)']
    
    # 情绪判断
    def get_sentiment(val):
        if val < 6000: return "🥶 冷清"
        if val < 8000: return "😐 温和"
        if val < 10000: return "😃 活跃"
        return "🔥 火爆"
    
    df['热度'] = df['总成交(亿)'].apply(get_sentiment)
    
    # 只取最近 N 天
    final_df = df.tail(days).copy()
    
    # --- 格式化显示 (解决间距问题) ---
    # 1. 设置日期格式为字符串
    final_df['日期'] = final_df['日期'].dt.strftime('%Y-%m-%d')
    
    # 2. 选取要显示的列
    cols = ['日期', '沪市(亿)', '深市(亿)', '北证(亿)', '总成交(亿)', '热度']
    display_df = final_df[cols]

    # 3. 利用 Pandas 的 Style 功能进行美化 (返回这个对象即可在 Notebook 中漂亮显示)
    # 对所有浮点数保留2位小数，并加上千分位分隔符
    styled_df = display_df.style.format({
        '沪市(亿)': '{:,.2f}',
        '深市(亿)': '{:,.2f}',
        '北证(亿)': '{:,.2f}',
        '总成交(亿)': '{:,.2f}'
    }).set_properties(**{
        'text-align': 'right', # 数字右对齐，方便对比
        'padding': '10px'      # 增加一点内边距
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]} # 表头居中
    ]).hide(axis='index') # 隐藏索引列(0,1,2...)

    return styled_df

# 运行并在 Notebook 中直接显示漂亮的表格
get_recent_market_turnover_fixed(7)

正在获取最近 7 天的全市场（修正版）成交数据...


日期,沪市(亿),深市(亿),北证(亿),总成交(亿),热度
2026-01-22,"12,017.64","14,899.68",249.15,"27,166.48",🔥 火爆
2026-01-23,"13,368.57","17,483.67",331.35,"31,183.59",🔥 火爆
2026-01-26,"14,518.60","17,963.43",328.14,"32,810.16",🔥 火爆
2026-01-27,"12,894.32","16,055.49",267.26,"29,217.07",🔥 火爆
2026-01-28,"13,655.26","15,998.84",272.01,"29,926.11",🔥 火爆
2026-01-29,"14,857.95","17,442.13",297.03,"32,597.12",🔥 火爆
2026-01-30,"12,698.56","15,656.82",272.11,"28,627.49",🔥 火爆


In [3]:
def analyze_top100_sectors_turnover():
    print("正在拉取最新成交额 Top 100 数据并分析板块流向...")
    
    # 东方财富排行接口
    url = "http://82.push2.eastmoney.com/api/qt/clist/get"
    params = {
        "pn": 1, "pz": 100, "po": 1, "np": 1, 
        "fltt": 2, "invt": 2,
        "fid": "f6", # 按成交额排序
        "fs": "m:0+t:6,m:0+t:80,m:1+t:2,m:1+t:23,m:0+t:81+s:2048", # 沪深北全市场
        "fields": "f12,f14,f6,f100,f3" # f12代码, f14名称, f6成交额, f100所属板块, f3涨跌幅
    }
    
    try:
        res = requests.get(url, params=params, headers={"User-Agent": "Mozilla/5.0"}).json()
        data = res['data']['diff']
        
        # 1. 构造基础 DataFrame
        df = pd.DataFrame(data)
        df = df.rename(columns={
            'f12': '代码', 
            'f14': '名称', 
            'f6': '成交额', 
            'f100': '所属板块', 
            'f3': '涨跌幅(%)'
        })
        
        # 2. 数据清洗
        df['成交额(亿)'] = df['成交额'] / 1e8
        # 某些新股可能没有板块数据，填充为'其他'
        df['所属板块'] = df['所属板块'].replace('', '其他/未分类')
        
        # 3. 【核心分析】按板块聚合统计
        # 统计指标：入围数量、板块内个股总成交、板块内个股平均涨跌
        sector_stats = df.groupby('所属板块').agg({
            '代码': 'count',
            '成交额(亿)': 'sum',
            '涨跌幅(%)': 'mean'
        }).rename(columns={'代码': '入围数量', '成交额(亿)': '聚合成交(亿)', '涨跌幅(%)': '平均涨跌(%)'})
        
        # 按“入围数量”和“聚合成交”降序排列
        sector_stats = sector_stats.sort_values(by=['入围数量', '聚合成交(亿)'], ascending=False)
        
        # 4. 格式化输出 - 板块统计表
        print("\n--- 📊 资金战场：Top 100 个股板块分布统计 ---")
        styled_sector = sector_stats.head(10).style.format({
            '聚合成交(亿)': '{:,.2f}',
            '平均涨跌(%)': '{:+.2f}%' # 显示正负号
        }).background_gradient(cmap='Reds', subset=['入围数量', '聚合成交(亿)']) # 颜色越深越热
        
        display(styled_sector) # 在 Notebook 中展示漂亮的表格
        
        # 5. 格式化输出 - 个股明细表 (前10名预览)
        print("\n--- 🔍 龙头详情：成交额排名前 10 个股 ---")
        top_stocks = df[['代码', '名称', '所属板块', '成交额(亿)', '涨跌幅(%)']].head(10)
        styled_stocks = top_stocks.style.format({
            '成交额(亿)': '{:,.2f}',
            '涨跌幅(%)': '{:+.2f}%'
        }).hide(axis='index')
        
        display(styled_stocks)
        
        return sector_stats, df

    except Exception as e:
        print(f"获取数据失败: {e}")
        return None, None

# 运行分析
sector_df, stock_df = analyze_top100_sectors_turnover()

正在拉取最新成交额 Top 100 数据并分析板块流向...

--- 📊 资金战场：Top 100 个股板块分布统计 ---


,入围数量,聚合成交(亿),平均涨跌(%)
所属板块,,,
半导体,14,997.54,+2.01%
通信设备,9,987.78,+7.72%
有色金属,8,728.19,-9.00%
小金属,7,491.91,-5.86%
互联网服务,7,474.58,-0.93%
文化传媒,4,374.62,-0.19%
消费电子,4,308.57,+0.74%
光伏设备,4,270.86,-1.29%
贵金属,4,266.86,-4.54%



--- 🔍 龙头详情：成交额排名前 10 个股 ---


代码,名称,所属板块,成交额(亿),涨跌幅(%)
601899,紫金矿业,有色金属,271.40,-7.62%
300308,中际旭创,通信设备,226.51,+5.73%
300058,蓝色光标,文化传媒,208.14,-0.30%
300502,新易盛,通信设备,207.65,+6.74%
300394,天孚通信,通信设备,175.78,+10.91%
300750,宁德时代,电池,170.49,+2.37%
688008,澜起科技,半导体,158.89,+12.13%
002155,湖南黄金,贵金属,134.14,+9.99%
600519,贵州茅台,酿酒行业,133.66,-2.55%
002131,利欧股份,互联网服务,123.82,-2.61%
